# Практика
Используемые библиотеки

In [ ]:
import random
import warnings
from itertools import combinations
from typing import List

import pandas as pd
import numpy as np
import requests
import io
import re
import urllib.request

from IPython.core.display_functions import display
%matplotlib inline

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [ ]:
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))
# df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')

In [ ]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
urllib.request.urlretrieve(url, "data-analysis.csv")

In [ ]:
df = pd.read_csv("data-analysis.csv")
df.set_index("Показатель", inplace=True)
df["Результат анализа"] = pd.to_numeric(df["Результат анализа"], errors="coerce")
df["Минимум"] = float("-inf")
df["Максимум"] = float("inf")

Можем добавить доп. вариант "не менее" как возможный

In [ ]:
# df.loc["Общая минерализация (сухой остаток)", "Норматив"] = "не менее 600"

In [ ]:
df

Заполним столбцы Максимум и Минимум

In [447]:
df.loc[df["Норматив"].str.match("не более"), "Максимум"] = df.loc[df["Норматив"].str.match("не более"), "Норматив"].str.findall(r'\d+,*\d*').apply(lambda x: x[-1].replace(',', '.')).astype(float).values

In [448]:
df.loc[df["Норматив"].str.match("не менее"), "Минимум"] = df.loc[df["Норматив"].str.match("не менее"), "Норматив"].str.findall(r'\d+,*\d*').apply(lambda x: x[0].replace(',', '.')).astype(float).values

In [449]:
df.loc[df["Норматив"].str.match("в пределах"), "Минимум"] = df.loc[df["Норматив"].str.match("в пределах"), "Норматив"].str.findall(r'\d+,*\d*').apply(lambda x: x[0].replace(',', '.')).astype(float).values

In [450]:
df.loc[df["Норматив"].str.match("в пределах"), "Максимум"] = df.loc[df["Норматив"].str.match("в пределах"), "Норматив"].str.findall(r'\d+,*\d*').apply(lambda x: x[-1].replace(',', '.')).astype(float).values

In [456]:
df["Вывод"] = df["Результат анализа"].isnull() | (df["Результат анализа"] < df["Максимум"]) & (df["Результат анализа"] > df["Минимум"])
df["Вывод"] = df["Вывод"].apply(lambda x: "Норма" if x else "Отклонение")

In [457]:
df

,Единица измерений,Результат анализа,Норматив,Минимум,Максимум,Вывод
Показатель,,,,,,
pH,единицы pH,8.400,в пределах 6-9,6.0,9.0,Норма
Запах,баллы,1.000,не более 2-3,-inf,3.0,Норма
Цветность,градусы,NaN,не более 30,-inf,30.0,Норма
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,7.0,10.0,Норма
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",-inf,1.5,Норма
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",-inf,3.3,Норма
Нитраты (по NO3),мг/дм3,24.000,не более 45,-inf,45.0,Норма
Фосфаты (P),мг/дм3,0.360,"не более 3,5",-inf,3.5,Норма
Хлориды (Cl),мг/дм3,200.000,не более 350,-inf,350.0,Норма


Заключаем что наблюдаются отклонения по Нефти и по окисляемости пермарганатной

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

Сначала приведем теоретическое решение. Вероятность это просто отношение количества сочетаний по 3 из 5 апельсинов ко всем сочетаням по 3 из 9

In [548]:
theory = len(list(combinations(range(5), 3))) / len(list(combinations(range(9), 3)))

Теперь проведем симуляцию

In [561]:
N = 100000
box = ['Я']*4 + ['A']*5
cases = []
def grab_fruit(box: List[int]):
    random.shuffle(box)
    return tuple(box[:3])

for i in range(N):
    cases.append(grab_fruit(box))

cases = pd.Series(cases)
simulator = len(cases[cases == ("A", "A", "A")]) / len(cases)

In [562]:
"{:.3%}".format(abs(simulator - theory) / theory)

'0.183%'

Погрешность в пределах 1 процента позволяет заключить что опыт релевантен теоретическим выкладкам

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?
В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он экспериментально?

Решим теоретически
У нас совокупность двух событий - мастер достал первую нестандартную и вторую стандартную. Только в этом случае он достанет ровно две детали. Перемножив вероятности этих событий получим ответ

In [566]:
theory = 7/10 * 3/9

0.23333333333333328

Проведем симуляцию

In [574]:
N = 100000
box = ['N']*3 + ['S']*7
cases = []
def grab_fruit(box: List[int]):
    random.shuffle(box)
    result = []
    for el in box:
        result.append(el)
        if el == 'S':
            return len(result)

for i in range(N):
    cases.append(grab_fruit(box))

cases = pd.Series(cases)
simulator = len(cases[cases == 2]) / len(cases)

In [575]:
"{:.3%}".format(abs(simulator - theory) / theory)

'0.083%'

Погрешность в пределах 1 процента позволяет заключить что опыт релевантен теоретическим выкладкам